In [292]:
import scipy.io
import numpy as np
import random
import time
from sklearn.cross_validation import KFold

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [224]:
a=time.time();b=a=time.time();a-b

0.0

In [3]:
mat = scipy.io.loadmat('/Users/yzh/Desktop/cour/supervised/HW1_Data/dataset1.mat')

In [9]:
mat.keys()

dict_keys(['Y_trn', 'X_tst', 'Y_tst', 'X_trn', '__header__', '__version__', '__globals__'])

In [164]:
def getPolyMat(input, power): #get poly feature matrix
  result=np.power(input,0)
  for i in range(1,power+1):
      result=np.concatenate((result,np.power(input,i)),axis=1)
  return result

In [169]:
mat['X_trn']

array([[ 1.41883768],
       [-3.08617234],
       [-2.33266533],
       [-2.38076152],
       [-0.53707415],
       [ 3.03807615],
       [ 2.66933868],
       [ 0.92184369],
       [ 2.49298597],
       [-3.90380762],
       [-2.09218437],
       [-0.2004008 ],
       [-2.90981964],
       [-1.8997996 ],
       [ 0.6492986 ],
       [-3.7755511 ],
       [-2.17234469],
       [ 3.34268537],
       [ 1.13026052],
       [-1.16232465],
       [ 0.90581162],
       [ 0.77755511],
       [ 0.29659319],
       [ 1.56312625],
       [ 1.498998  ],
       [-3.3747495 ],
       [ 1.06613226],
       [-3.74348697],
       [-2.34869739],
       [-2.42885772],
       [ 3.18236473],
       [-1.64328657],
       [ 2.55711423],
       [-1.35470942],
       [ 3.93587174],
       [-0.13627255],
       [-1.14629259],
       [ 1.22645291],
       [ 3.51903808],
       [-0.61723447],
       [-2.79759519],
       [-1.56312625],
       [ 1.32264529],
       [-1.94789579],
       [-3.35871743],
       [ 2

In [273]:
train_X=getPolyMat(mat['X_trn'], 3)

In [95]:
def closeFormRegression(train, label):
    return np.matmul(np.matmul( np.linalg.inv( np.matmul(train.transpose(), train) ),  train.transpose() ), label)

In [255]:
theta=closeFormRegression(train_X, mat['Y_trn'])

In [150]:
theta

array([[ 9.49203678],
       [ 4.79191663],
       [ 1.52906587]])

In [156]:
def GradDesc(train, label, learnRate, stopWhen, batchSize):# stochastic GD
    theta1=np.zeros((train.shape[1],1))
    while(True):
      batch=random.sample(range(0,label.shape[0]),batchSize)
      theta2 = theta1 - learnRate * np.matmul(train.transpose(), np.matmul(train, theta1)- label)
      if np.linalg.norm(theta2-theta1)<=stopWhen:
          return theta2
      theta1=theta2

In [157]:
GradDescTheta=GradDesc(train_X, mat['Y_trn'], 0.00001, 0.0001, 30)

In [290]:
print(np.linalg.norm( np.matmul(train_X,theta)-mat['Y_trn'] )/mat['Y_trn'].shape[0]**0.5)#train error

1.99198833363


In [274]:
test_X=getPolyMat(mat['X_tst'],3)

In [329]:
print(np.linalg.norm( np.matmul(test_X,theta)-mat['Y_tst'] )/mat['Y_tst'].shape[0]**0.5)#test error

7.35885642693


In [324]:
def get_error(train, theta, test): # get rmse( root mean square error)
    return np.linalg.norm( np.matmul(train,theta)-test )/test.shape[0]**0.5

In [195]:
np.matmul(test_X,GradDescTheta)-mat['Y_tst']

array([[  4.29493991e+00],
       [  9.21188757e+00],
       [  2.59443250e+01],
       [  1.72976566e+02],
       [ -8.83195394e+01],
       [ -4.16239499e+00],
       [  2.41985964e+01],
       [ -2.88429172e+01],
       [  1.73069874e+01],
       [  4.22130540e+00],
       [ -6.27556693e+00],
       [ -5.80134704e+00],
       [ -1.53802158e+01],
       [  1.35308453e+02],
       [  6.72211462e+01],
       [ -3.08746675e+00],
       [ -5.64855291e+00],
       [ -5.24551217e+01],
       [  1.14515305e+02],
       [  1.94482436e+02],
       [  1.83479282e+02],
       [  1.60472424e+02],
       [ -2.38893878e+01],
       [ -4.24280172e+01],
       [ -5.38114478e+00],
       [  3.66389470e+01],
       [ -4.45614036e+01],
       [ -2.00514055e+01],
       [ -4.04315192e+00],
       [  2.43473444e+00],
       [ -2.14390184e+01],
       [  2.39922363e+01],
       [  2.87063805e+01],
       [ -3.60257479e+00],
       [  1.07724086e+02],
       [  2.69313095e+01],
       [  7.70335542e+00],
 

In [275]:
def RidgeGradDesc(train, label, learnRate, stopWhen, batchSize,avoidOverFit):# stochastic GD
    theta1=np.zeros((train.shape[1],1))
    multiply=learnRate*2;
    while(True):
      batch=random.sample(range(0,label.shape[0]),batchSize)
      theta2 = theta1 - multiply * (np.matmul(train[batch,:].transpose(), np.matmul(train[batch,:],
                                    theta1)- label[batch,:])+avoidOverFit*theta1)
      if np.linalg.norm(theta2-theta1)<=stopWhen:
          return theta2
      theta1=theta2

In [280]:
GradDescTheta=RidgeGradDesc(train_X, mat['Y_trn'], 0.00000005, 0.0001, 30, 0.1)

In [284]:
def RidgecloseForm(train, label,avoidOverFit):# β=[(XtX+λI)^−1]XtY
    return np.matmul(np.matmul( np.linalg.inv( np.matmul(train.transpose(), 
             train) + avoidOverFit*np.identity(train.shape[1]) ), train.transpose() ), label)

In [327]:
theta=RidgecloseForm(train_X, mat['Y_trn'], 0.1)

In [297]:
Y_trn=mat['Y_trn']

In [321]:
foldNum=5

In [322]:
kf = KFold(train_X.shape[0], n_folds=foldNum)

In [366]:
pararid=(100,10, 1,0.1,0.01, 0.001, 0.0001,0)
#optAvoidOverFit=pararid[0]
paraErrorDict={}
for avoidOverFit in pararid:
  sum=0;
  count=0;
  for train_index, test_index in kf:
    Xtrain, Xtest = train_X[train_index], train_X[test_index]
    ytrain, ytest = Y_trn[train_index], Y_trn[test_index]
    sum+=get_error(Xtest,RidgecloseForm(Xtrain, ytrain,avoidOverFit),ytest);  count+=1
  paraErrorDict[avoidOverFit]=sum/count

In [373]:
pararid=(100,10, 1,0.1,0.01, 0.001, 0.0001,0)
#optAvoidOverFit=pararid[0]
paraErrorDict={}
for avoidOverFit in pararid:
  sum=0;
  count=0;
  for train_index, test_index in kf:
    Xtrain, Xtest = train_X[train_index], train_X[test_index]
    ytrain, ytest = Y_trn[train_index], Y_trn[test_index]
    sum+=get_error(Xtest,RidgeGradDesc(Xtrain, ytrain,0.00000005, 0.0001,30,avoidOverFit),ytest);  count+=1
  paraErrorDict[avoidOverFit]=sum/count

In [372]:
optAvoidOverFit

0

In [380]:
paraErrorDict

{0: 9.7944525466551298,
 0.0001: 9.7941036005048296,
 0.001: 10.538072248155263,
 0.01: 10.14803283594015,
 0.1: 9.6554209752079778,
 1: 10.091720753204244,
 10: 9.6364980091535681,
 100: 10.53950746642079}

In [377]:
optAvoidOverFit=pararid[0]
for i in range(1, len(pararid)):
    if paraErrorDict[pararid[i]]<paraErrorDict[optAvoidOverFit]:
        optAvoidOverFit=pararid[i]

In [379]:
optAvoidOverFit

10